## Machine Learning

### Importación de las librerías necesarias

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Se utilizarán las librerías sklearn.feature_extraction.text y sklearn.metrics.pairwise para implementar un sistema de recomendación basado en filtrado de contenido en el que se compararán las similitudes entre diferentes elementos de un conjunto de datos. La librería CountVectorizer se utilizará para convertir los datos de texto en una matriz numérica, mientras que cosine_similarity se utilizará para calcular la similitud coseno entre los elementos de la matriz. Esto permitirá generar recomendaciones personalizadas basadas en los patrones de similitud de los datos de entrada.

### Carga de datos

In [3]:
df=pd.read_csv(r"https://raw.githubusercontent.com/JersonGB22/ProyectoIndividual1_MLOps_Henry/main/Datasets/ML_movie_transformation.csv")
df

,title,label
0,toy story,toy stori anim comedi famili tom hank tim alle...
1,jumanji,jumanji adventur fantasi famili robin william ...
2,grumpier old men,grumpier old men romanc comedi walter matthau ...
3,waiting to exhale,wait exhal comedi drama romanc whitney houston...
4,father of the bride part ii,father bride part ii comedi steve martin dian ...
...,...,...
21803,the sublet,sublet mysteri drama thriller horror tianna no...
21804,cadet kelly,cadet kelli comedi hilari duff christi carlson...
21805,pooh's heffalump halloween movie,pooh heffalump halloween movi anim famili jimm...
21806,mom,mom crime drama thriller sridevi kapoor sajal ...


### Matriz de Caracaterísticas

In [4]:
# Visualización del registro con mayor cantidad de palabras
word=df.label.str.split().str.len()
print("Número de Palabras: ",word.max())
df.label.iloc[word.idxmax()]

Número de Palabras:  686


"around world eighti day adventur comedi famili fantasi romanc david niven cantinfla finlay curri robert morley charl boyer robert newton gilbert roland marlen dietrich frank sinatra buster keaton john carradin peter lorr georg raft tim mccoy joe e. brown shirley maclain melvil cooper reginald denni ronald colman trevor howard harcourt william martin carol cedric hardwick noël coward john gielgud fernandel evelyn key josé greco cesar romero alan mowbray charl coburn red skelton andi devin edmund low victor mclaglen john mill glyni john hermion gingold edward r. murrow mike mazurki ronald squir basil sydney lui miguel dominguín robert cabal jack oaki beatric lilli a.e. matthew walter fitzgerald ronald adam frank royd abdullah abba jess adam fred aldrich ray armstrong gertrud astor walter bacon rama bai leah baird brandon beach eugen beday helena benda audrey betz georg blagoi eumenio blanco nina borget danni borzag hazel boyn georg bruggeman bob burrow paul busch gordon carveth spencer 

In [5]:
# Visualización del registro con menor cantidad de palabras
print("Número de Palabras: ",word.min())
df.label.iloc[word.idxmin()]

Número de Palabras:  6


'food chain documentari sanjay rawal english'

In [6]:
cv=CountVectorizer(max_features=1000,stop_words="english")
vector=cv.fit_transform(df.label).toarray()

* Al utilizar max_features=1000, estoy seleccionando un número cercano al doble del máximo de palabras presentes en mis registros, que es de 686. Esta elección me permite capturar un rango más amplio de palabras relevantes en mis datos sin agregar un exceso de características irrelevantes. Al limitar el número de características a 1000, puedo mantener un equilibrio entre la representatividad de las palabras seleccionadas y la eficiencia del modelo. Además, al eliminar las palabras comunes en inglés con stop_words="english", estoy asegurando que las características restantes sean más significativas y relevantes para mi modelo, lo que puede mejorar la calidad de las predicciones y reducir el ruido en los resultados obtenidos.

In [7]:
# Visualizando el array vector
print(vector.shape)
vector

(21808, 1000)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### Matriz de Similitud

In [8]:
similarity_score=cosine_similarity(vector)

In [9]:
# Visualizando el array similarity_score
print(similarity_score.shape)
similarity_score

(21808, 21808)


array([[1.        , 0.12930164, 0.20890726, ..., 0.23745309, 0.02865341,
        0.24770754],
       [0.12930164, 1.        , 0.13055824, ..., 0.18212505, 0.10744306,
        0.18999015],
       [0.20890726, 0.13055824, 1.        , ..., 0.20343337, 0.03857584,
        0.21221867],
       ...,
       [0.23745309, 0.18212505, 0.20343337, ..., 1.        , 0.0717496 ,
        0.25374774],
       [0.02865341, 0.10744306, 0.03857584, ..., 0.0717496 , 1.        ,
        0.07484812],
       [0.24770754, 0.18999015, 0.21221867, ..., 0.25374774, 0.07484812,
        1.        ]])

* Crearemos una función que se utilizará para indexar las películas más similares a una película específica en el conjunto de datos. Dado un título de película, la función encuentra su índice correspondiente en el conjunto de datos y utiliza la matriz de similitud previamente calculada, para determinar los 5 elementos más similares a la película de referencia. La función devuelve una lista de índices correspondientes a las películas más similares en orden descendente, lo que permite utilizar esta información para hacer recomendaciones personalizadas en el endpoint 7 de la API de manera más rápida, eficiente y efectiva.

In [10]:
def index_movie(titulo:str):
    index=df[df.title==titulo].index[0]
    score=list(enumerate(similarity_score[index]))
    score=sorted(score,key=lambda x: x[1],reverse=True)
    # El índice 0 siempre corresponde a la misma película, por lo cual, no se considera
    score=score[1:6]
    return [i[0] for i in score]

In [11]:
df["index_movie"]=df.title.apply(index_movie)
df

,title,label,index_movie
0,toy story,toy stori anim comedi famili tom hank tim alle...,"[2285, 10998, 11507, 11250, 6278]"
1,jumanji,jumanji adventur fantasi famili robin william ...,"[10720, 13024, 7785, 20185, 1472]"
2,grumpier old men,grumpier old men romanc comedi walter matthau ...,"[2530, 19874, 9685, 3117, 3161]"
3,waiting to exhale,wait exhal comedi drama romanc whitney houston...,"[519, 4133, 8234, 6340, 7862]"
4,father of the bride part ii,father bride part ii comedi steve martin dian ...,"[5130, 15594, 21235, 10764, 20713]"
...,...,...,...
21803,the sublet,sublet mysteri drama thriller horror tianna no...,"[7632, 14641, 16555, 19489, 7688]"
21804,cadet kelly,cadet kelli comedi hilari duff christi carlson...,"[20118, 1124, 7661, 21375, 2928]"
21805,pooh's heffalump halloween movie,pooh heffalump halloween movi anim famili jimm...,"[7291, 2415, 13753, 10539, 2950]"
21806,mom,mom crime drama thriller sridevi kapoor sajal ...,"[15582, 19492, 19862, 18107, 20663]"


### Exportación del df_movie para la realización del endpoint de ML (función 7)

In [16]:
# La columna "label" ya no será necesaria
df.drop(columns=["label"]).to_csv("API_ML_movie.csv",index=False)

---